1)Если еще не сделано, то: Модифицировать приложение из предыдущего домашнего задания:
- приложение должно собирать вакансии сразу с двух сайтов hh.ru и superjob.ru
- собранные данные должны быть приведены к общей структуре
- разделить зарплату на две составляющие (мин. и макс.) и сохранить в виде int. Если валюта указана другая, привести все к рублям.

2)Реализовать сохранение полученных вакансий в СУБД (на выбор SQLite или MongoDB)
3)Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы.
3*)Модифицировать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [176]:
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

#### Загрузка датасетов

In [177]:
HH_PATH = 'hh программист с++.csv'
SUPERJOB_PATH = 'superjob программист c#.csv'

In [178]:
hh_df = pd.read_csv(HH_PATH, encoding='utf-8')
hh_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Программист С++/Qt,70000.0,NaN,https://hh.ru/vacancy/31273030?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
1,Программист С++ (графика),80000.0,NaN,https://hh.ru/vacancy/32543357?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
2,Программист С++,NaN,100000.0,https://hh.ru/vacancy/32417827?query=%D0%BF%D1...,hh.ru,Группа Компаний «АМЕ»


In [179]:
sj_df = pd.read_csv(SUPERJOB_PATH, encoding='utf-8')
sj_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Инженер-программист / Разработчик C#,300000.0,NaN,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,ФГУ НПП Гамма
1,Инженер-программист C#,NaN,NaN,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,Центральная ППК
2,.NET Developer / Разработчик C#,NaN,NaN,https://www.superjob.ru/vakansii/net-developer...,superjob.ru,Информационные технологии


In [180]:
hh_df.dtypes

vacansy        object
min_salary    float64
max_salary    float64
url            object
source         object
employer       object
dtype: object

In [181]:
sj_df.dtypes

vacansy        object
min_salary    float64
max_salary    float64
url            object
source         object
employer       object
dtype: object

#### Заполним NaN значения нулями

In [182]:
values = {'min_salary': 0, 'max_salary': 0}
hh_df = hh_df.fillna(value=values,);
sj_df = sj_df.fillna(value=values);

In [183]:
hh_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Программист С++/Qt,70000.0,0.0,https://hh.ru/vacancy/31273030?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
1,Программист С++ (графика),80000.0,0.0,https://hh.ru/vacancy/32543357?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
2,Программист С++,0.0,100000.0,https://hh.ru/vacancy/32417827?query=%D0%BF%D1...,hh.ru,Группа Компаний «АМЕ»


In [184]:
sj_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Инженер-программист / Разработчик C#,300000.0,0.0,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,ФГУ НПП Гамма
1,Инженер-программист C#,0.0,0.0,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,Центральная ППК
2,.NET Developer / Разработчик C#,0.0,0.0,https://www.superjob.ru/vakansii/net-developer...,superjob.ru,Информационные технологии


#### Подключаемся к БД и создаем записи

In [185]:
client = MongoClient('localhost',27017)
db = client['vacancys']

#### Вставляем данные в БД

In [186]:
hh_db.delete_many({})
sj_db.delete_many({})

In [187]:
data = hh_df.to_dict(orient='records')
hh_db = db.hh_cpp
hh_db.insert_many(data)

In [188]:
hh_db.count()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


165

In [189]:
data = sj_df.to_dict(orient='records')
sj_db = db.sj_csharp
sj_db.insert_many(data)

In [190]:
sj_db.count()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


200

#### Функция поиска и вывода на экран вакансий с суммой, вводимой пользователем

In [162]:
def showVacancys(col_name):
    salary = input('Введите сумму: ')
    for post in col_name.find({'max_salary': {'$gt':int(salary)}}):
        pprint(post)

#### Демонстрация работы функции

In [163]:
showVacancys(hh_db)

Введите сумму: 150000
{'_id': ObjectId('5d8106f4a88bb05903421bbf'),
 'employer': ' ФОРМ',
 'max_salary': 170000.0,
 'min_salary': 115000.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/33289931?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Ведущий инженер-программист С++'}
{'_id': ObjectId('5d8106f4a88bb05903421bd3'),
 'employer': ' Никс',
 'max_salary': 170000.0,
 'min_salary': 115000.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/32985391?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Программист С/C++'}
{'_id': ObjectId('5d8106f4a88bb05903421bfb'),
 'employer': ' BeeJee',
 'max_salary': 170000.0,
 'min_salary': 115000.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/32455941?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Web-разработчик (Node.js, React)'}
{'_id': ObjectId('5d8106f4a88bb05903421c37')

In [164]:
showVacancys(sj_db)

Введите сумму: 150000
{'_id': ObjectId('5d810707a88bb05903421d27'),
 'employer': 'Минерал-Инфо',
 'max_salary': 200000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/web-programmist-29723693.html',
 'vacansy': 'Web-программист / Web-разработчик'}
{'_id': ObjectId('5d810707a88bb05903421d2b'),
 'employer': 'Кадровый Центр на Сокольнической Площади',
 'max_salary': 250000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/razrabotchik-c-32357613.html',
 'vacansy': 'Разработчик C#'}
{'_id': ObjectId('5d810707a88bb05903421d2c'),
 'employer': '«Группа ГУТА»',
 'max_salary': 190000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/full-stack-developer-32491461.html',
 'vacansy': 'Full-Stack developer (Angular, C#, .NET)'}
{'_id': ObjectId('5d810707a88bb05903421d3b'),
 'employer': 'Минерал-Инфо',
 'max_salary': 200000.0,
 'min_salary': 150000.0,
 'source':

#### Создадим еще один датафрейм и приклеим его к старому

In [191]:
hh_df2 = pd.DataFrame({'vacansy':['new_vacancy','new_vacancy2'], 
                       'min_salary':[999999,999999],
                       'max_salary':[999999,999999],
                       'url':['new_url','new_url2'],
                       'source':['hh.ru','hh.ru'],
                       'employer':['new_employer','new_employer2']}) 

In [192]:
hh_df2.head(10)

,vacansy,min_salary,max_salary,url,source,employer
0,new_vacancy,999999,999999,new_url,hh.ru,new_employer
1,new_vacancy2,999999,999999,new_url2,hh.ru,new_employer2


In [193]:
hh_df = hh_df.append(hh_df2, ignore_index = True) 

In [194]:
hh_df.tail()

,vacansy,min_salary,max_salary,url,source,employer
162,Системный программист в отдел адаптации Sailfi...,0.0,0.0,https://hh.ru/vacancy/29727812?query=%D0%BF%D1...,hh.ru,Открытая Мобильная Платформа
163,Старший математик-программист,0.0,0.0,https://hh.ru/vacancy/27444585?query=%D0%BF%D1...,hh.ru,НСК
164,Архитектор 1С УХ,80.0,0.0,https://hh.ru/vacancy/33077576?query=%D0%BF%D1...,hh.ru,ООО Вератис
165,new_vacancy,999999.0,999999.0,new_url,hh.ru,new_employer
166,new_vacancy2,999999.0,999999.0,new_url2,hh.ru,new_employer2


In [201]:
sj_df2 = pd.DataFrame({'vacansy':['new_vacancy','new_vacancy2'], 
                       'min_salary':[999999,999999],
                       'max_salary':[999999,999999],
                       'url':['new_url','new_url2'],
                       'source':['superjob.ru','superjob.ru'],
                       'employer':['new_employer','new_employer2']}) 

In [202]:
sj_df = sj_df.append(sj_df2, ignore_index = True) 

In [203]:
sj_df.tail()

,vacansy,min_salary,max_salary,url,source,employer
197,Full-Stack developer (разработка корпоративных...,0.0,0.0,https://www.superjob.ru/vakansii/full-stack-de...,superjob.ru,«Группа ГУТА»
198,Разработчик MS SQL,0.0,0.0,https://www.superjob.ru/vakansii/razrabotchik-...,superjob.ru,Сбербанк России
199,Программист .Net (full-stack),0.0,0.0,https://www.superjob.ru/vakansii/programmist-3...,superjob.ru,НПП Гарант Сервис Университет
200,new_vacancy,999999.0,999999.0,new_url,superjob.ru,new_employer
201,new_vacancy2,999999.0,999999.0,new_url2,superjob.ru,new_employer2


#### Функция обновления базы данных

In [195]:
def col_update(col_name, df):
    data = df.to_dict(orient='records')
    for i,item in enumerate(data): 
        col_name.update_one({'vacansy': data[i]['vacansy']  ,
                             'min_salary': data[i]['min_salary']  ,
                             'max_salary': data[i]['max_salary']  ,
                             'url': data[i]['url']  ,
                             'source': data[i]['source']  ,
                             'employer': data[i]['employer'] },
                            {'$set': { 'vacansy' : data[i]['vacansy'],
                                       'min_salary' : data[i]['min_salary'],
                                       'max_salary' : data[i]['max_salary'],
                                       'url' : data[i]['url'],
                                       'source' : data[i]['source'],
                                       'employer' : data[i]['employer']
                                     }
                            } ,upsert=True)

#### Обновим коллекции и посмотрим добавились ли наши вакансии

In [198]:
col_update(hh_db, hh_df)

In [200]:
showVacancys(hh_db)

Введите сумму: 999000
{'_id': ObjectId('5d81099599dae9ef1e8fcfa1'),
 'employer': 'new_employer',
 'max_salary': 999999.0,
 'min_salary': 999999.0,
 'source': 'hh.ru',
 'url': 'new_url',
 'vacansy': 'new_vacancy'}
{'_id': ObjectId('5d81099599dae9ef1e8fcfa3'),
 'employer': 'new_employer2',
 'max_salary': 999999.0,
 'min_salary': 999999.0,
 'source': 'hh.ru',
 'url': 'new_url2',
 'vacansy': 'new_vacancy2'}


In [204]:
col_update(sj_db, sj_df)

In [205]:
showVacancys(sj_db)

Введите сумму: 999000
{'_id': ObjectId('5d810a5099dae9ef1e8fd165'),
 'employer': 'new_employer',
 'max_salary': 999999.0,
 'min_salary': 999999.0,
 'source': 'superjob.ru',
 'url': 'new_url',
 'vacansy': 'new_vacancy'}
{'_id': ObjectId('5d810a5099dae9ef1e8fd167'),
 'employer': 'new_employer2',
 'max_salary': 999999.0,
 'min_salary': 999999.0,
 'source': 'superjob.ru',
 'url': 'new_url2',
 'vacansy': 'new_vacancy2'}
